In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }
)

### Connect with databse and ask basic questions

In [282]:
from langchain_community.utilities import SQLDatabase

db_user = "root"
db_password = ""
db_host = "localhost"
db_name = "Pharma"
db_port = 3307

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)



CREATE TABLE customers (
	`CustomerID` INTEGER(11) NOT NULL, 
	`Name` VARCHAR(255), 
	`Address` TEXT, 
	`ContactInfo` VARCHAR(255), 
	PRIMARY KEY (`CustomerID`)
)ENGINE=MyISAM COLLATE latin1_swedish_ci DEFAULT CHARSET=latin1

/*
3 rows from customers table:
CustomerID	Name	Address	ContactInfo
1	John Doe	101 Main St, Anytown, USA	john.doe@example.com
2	Jane Smith	102 Pine Rd, Somewhere, USA	jane.smith@example.com
3	Michael Johnson	103 Oak St, Thisplace, USA	michael.johnson@example.com
*/


CREATE TABLE inventory (
	`InventoryID` INTEGER(11) NOT NULL, 
	`MedicineID` INTEGER(11), 
	`SupplierID` INTEGER(11), 
	`Quantity` INTEGER(11), 
	`BatchNo` VARCHAR(255), 
	`ExpiryDate` DATE, 
	PRIMARY KEY (`InventoryID`)
)ENGINE=MyISAM COLLATE latin1_swedish_ci DEFAULT CHARSET=latin1

/*
3 rows from inventory table:
InventoryID	MedicineID	SupplierID	Quantity	BatchNo	ExpiryDate
1	1	1	150	B1234	2025-09-30
2	2	2	200	C2345	2024-10-15
3	3	3	120	D3456	2026-01-20
*/


CREATE TABLE medicines (
	`MedicineID` 

In [181]:
db.get_table_info

<bound method SQLDatabase.get_table_info of <langchain_community.utilities.sql_database.SQLDatabase object at 0x000001ACB28A2D90>>

In [17]:
db.run("SELECT * FROM CUSTOMERS LIMIT 10;")

"[(1, 'John Doe', '101 Main St, Anytown, USA', 'john.doe@example.com'), (2, 'Jane Smith', '102 Pine Rd, Somewhere, USA', 'jane.smith@example.com'), (3, 'Michael Johnson', '103 Oak St, Thisplace, USA', 'michael.johnson@example.com'), (4, 'Emily Davis', '104 Maple Ave, Thatplace, USA', 'emily.davis@example.com'), (5, 'Daniel Martinez', '105 Elm St, Anywhere, USA', 'daniel.martinez@example.com'), (6, 'Lucy Garcia', '106 Cedar Rd, Somewhere Else, USA', 'lucy.garcia@example.com'), (7, 'Ethan Brown', '107 Birch St, Overthere, USA', 'ethan.brown@example.com'), (8, 'Sophia Wilson', '108 Ash Ave, Right Here, USA', 'sophia.wilson@example.com'), (9, 'Mason Miller', '109 Pine St, Left There, USA', 'mason.miller@example.com'), (10, 'Olivia Moore', '110 Spruce Rd, Nowhere, USA', 'olivia.moore@example.com')]"

In [283]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

In [27]:
chain.get_prompts()[0].template

'You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use CURDATE() function to get the current date, if the question involves "today".\n\nUse the following format:\n\nQuestion: Question here\nSQLQuery: SQL Query to run\nSQLResult

In [76]:
# from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
# from langchain_core.output_parsers import StrOutputParser

# execute_query = QuerySQLDataBaseTool(db=db)
# write_query = create_sql_query_chain(llm, db)
# chain = write_query | extract_sql | execute_query | StrOutputParser()
# result = chain.invoke({"question": "List all medicines that have less than 200 units in stock."})

# print(result.strip())


[('Amlodipine',), ('Amoxicillin',), ('Aspirin',), ('Atorvastatin',), ('Azithromycin',)]


### Few shot learning examples

In [286]:
few_shots = [
    {
        'Question': "What is the total quantity of Amoxicillin in stock?",
        'SQLQuery': "SELECT SUM(Quantity) FROM Inventory JOIN Medicines ON Inventory.MedicineID = Medicines.MedicineID WHERE Medicines.Name = 'Amoxicillin'",
        'SQLResult': "Result of the SQL query",
        'Answer': "320"
    },
    {
        'Question': "How many different medicines are supplied by 'PharmaSupply Co.'?",
        'SQLQuery': "SELECT COUNT(DISTINCT MedicineID) FROM Inventory JOIN Suppliers ON Inventory.SupplierID = Suppliers.SupplierID WHERE Suppliers.Name = 'PharmaSupply Co.'",
        'SQLResult': "Result of the SQL query",
        'Answer': "5"
    },
    {
        'Question': "What is the total sales amount for February 2024?",
        'SQLQuery': "SELECT SUM(TotalAmount) FROM Sales WHERE SaleDate BETWEEN '2024-02-01' AND '2024-02-28'",
        'SQLResult': "Result of the SQL query",
        'Answer': "1543.75"
    },
    {
        'Question': "List all medicines that have less than 200 units in stock.",
        'SQLQuery': "SELECT Medicines.Name FROM Inventory JOIN Medicines ON Inventory.MedicineID = Medicines.MedicineID WHERE Inventory.Quantity < 200",
        'SQLResult': "Result of the SQL query",
        'Answer': "Medicine names with less than 50 units in stock"
    },
    {
        'Question': "Which customer made the most purchases in January 2024?",
        'SQLQuery': "SELECT Customers.Name FROM Sales JOIN Customers ON Sales.CustomerID = Customers.CustomerID WHERE SaleDate BETWEEN '2024-01-01' AND '2024-01-31' GROUP BY Sales.CustomerID ORDER BY SUM(Sales.Quantity) DESC LIMIT 1",
        'SQLResult': "Result of the SQL query",
        'Answer': "Customer name with the most purchases"
    },
    {
        'Question': "How many unique suppliers are there for the medicine 'Lisinopril'?",
        'SQLQuery': "SELECT COUNT(DISTINCT SupplierID) FROM Inventory JOIN Medicines ON Inventory.MedicineID = Medicines.MedicineID WHERE Medicines.Name = 'Lisinopril'",
        'SQLResult': "Result of the SQL query",
        'Answer': "3"
    },
    {
        'Question': "What is the average sale amount per transaction in March 2024?",
        'SQLQuery': "SELECT AVG(TotalAmount) FROM Sales WHERE SaleDate BETWEEN '2024-03-01' AND '2024-03-31'",
        'SQLResult': "Result of the SQL query",
        'Answer': "27.89"
    },
    {
        'Question': "Find the name and contact information of customers who have purchased 'Metformin' more than once.",
        'SQLQuery': "SELECT DISTINCT Customers.Name, Customers.ContactInfo FROM Sales JOIN Customers ON Sales.CustomerID = Customers.CustomerID JOIN Medicines ON Sales.MedicineID = Medicines.MedicineID WHERE Medicines.Name = 'Metformin' GROUP BY Sales.CustomerID HAVING COUNT(Sales.SaleID) > 1",
        'SQLResult': "Result of the SQL query",
        'Answer': "Customer names and contact information"
    },
    {
        'Question': "List all customers who have not made any purchases in the last 3 months.",
        'SQLQuery': "SELECT Customers.Name FROM Customers LEFT JOIN Sales ON Customers.CustomerID = Sales.CustomerID WHERE Sales.SaleDate <= DATE_SUB(NOW(), INTERVAL 3 MONTH) OR Sales.SaleID IS NULL",
        'SQLResult': "Result of the SQL query",
        'Answer': "List of customers who have not made any purchases"
    },
    {
        'Question': "Find the top 5 best-selling medicines in terms of total revenue generated.",
        'SQLQuery': "SELECT Medicines.Name, SUM(Sales.Quantity * Sales.TotalAmount) AS Revenue FROM Sales JOIN Medicines ON Sales.MedicineID = Medicines.MedicineID GROUP BY Medicines.MedicineID ORDER BY Revenue DESC LIMIT 5",
        'SQLResult': "Result of the SQL query",
        'Answer': "Top 5 best-selling medicines by revenue"
    },
    {
        'Question': "Calculate the average quantity of medicines purchased by each customer.",
        'SQLQuery': "SELECT Customers.Name, AVG(Sales.Quantity) AS AvgQuantity FROM Sales JOIN Customers ON Sales.CustomerID = Customers.CustomerID GROUP BY Customers.CustomerID",
        'SQLResult': "Result of the SQL query",
        'Answer': "Average quantity of medicines purchased by each customer"
    },
    {
        'Question': "List all medicines that are about to expire within the next month, sorted by expiry date.",
        'SQLQuery': "SELECT Medicines.Name, Inventory.ExpiryDate FROM Inventory JOIN Medicines ON Inventory.MedicineID = Medicines.MedicineID WHERE Inventory.ExpiryDate BETWEEN NOW() AND DATE_ADD(NOW(), INTERVAL 1 MONTH) ORDER BY Inventory.ExpiryDate",
        'SQLResult': "Result of the SQL query",
        'Answer': "List of medicines about to expire within the next month"
    },
    {
        'Question': "Find the total sales amount for each month of the year 2024.",
        'SQLQuery': "SELECT DATE_FORMAT(SaleDate, '%Y-%m') AS Month, SUM(TotalAmount) AS TotalSales FROM Sales WHERE YEAR(SaleDate) = 2024 GROUP BY Month",
        'SQLResult': "Result of the SQL query",
        'Answer': "Total sales amount for each month of 2024"
    },
    {
        'Question': "Identify the suppliers who have supplied all available medicines.",
        'SQLQuery': "SELECT Suppliers.Name FROM Suppliers WHERE NOT EXISTS (SELECT DISTINCT MedicineID FROM Medicines WHERE NOT EXISTS (SELECT MedicineID FROM Inventory WHERE Inventory.MedicineID = Medicines.MedicineID AND Inventory.SupplierID = Suppliers.SupplierID))",
        'SQLResult': "Result of the SQL query",
        'Answer': "Suppliers who have supplied all available medicines"
    }
]

### Creating Semantic Similarity Based Example Selector

In [287]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings()
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [288]:
to_vectorize

["What is the total quantity of Amoxicillin in stock? SELECT SUM(Quantity) FROM Inventory JOIN Medicines ON Inventory.MedicineID = Medicines.MedicineID WHERE Medicines.Name = 'Amoxicillin' Result of the SQL query 320",
 "How many different medicines are supplied by 'PharmaSupply Co.'? SELECT COUNT(DISTINCT MedicineID) FROM Inventory JOIN Suppliers ON Inventory.SupplierID = Suppliers.SupplierID WHERE Suppliers.Name = 'PharmaSupply Co.' Result of the SQL query 5",
 "What is the total sales amount for February 2024? SELECT SUM(TotalAmount) FROM Sales WHERE SaleDate BETWEEN '2024-02-01' AND '2024-02-28' Result of the SQL query 1543.75",
 'List all medicines that have less than 200 units in stock. SELECT Medicines.Name FROM Inventory JOIN Medicines ON Inventory.MedicineID = Medicines.MedicineID WHERE Inventory.Quantity < 200 Result of the SQL query Medicine names with less than 50 units in stock',
 "Which customer made the most purchases in January 2024? SELECT Customers.Name FROM Sales JOI

In [289]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [290]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many totall no of customers?"})

[{'Answer': 'Average quantity of medicines purchased by each customer',
  'Question': 'Calculate the average quantity of medicines purchased by each customer.',
  'SQLQuery': 'SELECT Customers.Name, AVG(Sales.Quantity) AS AvgQuantity FROM Sales JOIN Customers ON Sales.CustomerID = Customers.CustomerID GROUP BY Customers.CustomerID',
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'Average quantity of medicines purchased by each customer',
  'Question': 'Calculate the average quantity of medicines purchased by each customer.',
  'SQLQuery': 'SELECT Customers.Name, AVG(Sales.Quantity) AS AvgQuantity FROM Sales JOIN Customers ON Sales.CustomerID = Customers.CustomerID GROUP BY Customers.CustomerID',
  'SQLResult': 'Result of the SQL query'}]

### Setting up PromptTemplate

In [291]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
If the question explicitly asks to list down all relevant entries without limiting them, do not use the LIMIT clause, and retrieve all matching entries.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [292]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [293]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [294]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [302]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser

execute_query = QuerySQLDataBaseTool(db=db)

query_chain = create_sql_query_chain(llm, db, prompt=few_shot_prompt)
query = query_chain.invoke({"question": "list down all the medicines names that are more than 100 in quantity"})

print(query)
result = execute_query(query)
print(result)


SELECT Medicines.Name FROM Inventory JOIN Medicines ON Inventory.MedicineID = Medicines.MedicineID WHERE Inventory.Quantity > 100
[('Amoxicillin',), ('Paracetamol',), ('Lisinopril',), ('Atorvastatin',), ('Simvastatin',), ('Cetirizine',), ('Ibuprofen',), ('Naproxen',), ('Prednisone',), ('Omeprazole',), ('Fluoxetine',), ('Warfarin',), ('Insulin Glargine',)]


In [303]:
def natural_language_response(result):
    prompt_template = """
    Based on the query and result, write a natural language response:
    Here are the queries and results from the latest database query:

    SQL Query : {query}
    Results : {result}
    """

    prompt = prompt_template.format(query=query, result=result)

    response = llm.invoke(prompt)

    return response.content

response = natural_language_response(result)

print(response)

Based on the database query and the results obtained, we can extract the following information:

1. The query was executed on two tables: "Inventory" and "Medicines."

2. The query retrieved the "Name" column from the "Medicines" table.

3. The query joined the "Inventory" and "Medicines" tables on the "MedicineID" column, which is common between them.

4. The query applied a filter condition to only include records from the "Inventory" table where the "Quantity" column is greater than 100.

5. The results of the query returned 13 rows, which are the names of medicines that have a quantity greater than 100 in the inventory. These medicines are:
   - Amoxicillin
   - Paracetamol
   - Lisinopril
   - Atorvastatin
   - Simvastatin
   - Cetirizine
   - Ibuprofen
   - Naproxen
   - Prednisone
   - Omeprazole
   - Fluoxetine
   - Warfarin
   - Insulin Glargine

These results provide insights into the medicines that are adequately stocked in the inventory, with quantities exceeding 100. This 

In [305]:
import ast
import pandas as pd

def process_result(result):
    try:
        if not result:
            print("No results found...")
            return

        correct_result = ast.literal_eval(result)
        
        if isinstance(correct_result, list) and all(isinstance(row, tuple) for row in correct_result):
            df = pd.DataFrame(correct_result)
            response = df.to_string(header=False, index=False)
            return response
        else:
            response = natural_language_response(result)
            return response
    except Exception as e:
        return ("An error occurred:", e)

output = process_result(result)
print(output)

     Amoxicillin
     Paracetamol
      Lisinopril
    Atorvastatin
     Simvastatin
      Cetirizine
       Ibuprofen
        Naproxen
      Prednisone
      Omeprazole
      Fluoxetine
        Warfarin
Insulin Glargine
